# Instalação e Bibliotecas

In [ ]:
# Instalar a biblioteca Python google-genai
!pip -q install google-genai

In [ ]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings
import os
from google.colab import userdata
from google import genai
from IPython.display import HTML, Markdown

In [ ]:
# Para ignorar todas as mensagens de aviso que o Python possa gerar durante a execução do código
warnings.filterwarnings("ignore")

In [ ]:
# Configura a API Key do Google Gemini
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini
client = genai.Client()

modelo = "gemini-2.0-flash"

# Funções Auxiliares

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
  minha_localizacao = ""
  while not minha_localizacao.strip(): # Loop até que uma localização seja fornecida
      minha_localizacao_input = input("\nPor favor, digite a cidade e o estado em que você está (ex: Maringá, Paraná): ")
      if minha_localizacao_input and minha_localizacao_input.strip():
          minha_localizacao = minha_localizacao_input.strip()
      else:
          print("Localização inválida. Por favor, tente novamente.")
  print(f"Localização definida: {minha_localizacao}")

# Criação dos Agentes

## Agente 1 - Selecionador de produto

In [ ]:
# Lista de produtos para descarte consciente
produtos = [
    "Plástico", "Papel", "Metal", "Lâmpadas", "Móveis", "Vidro",
    "Roupa velha", "Esponja de cozinha", "Pilhas e baterias",
    "Eletrônicos", "Óleo de cozinha usado", "Medicamentos vencidos"
]

In [ ]:
def selecionar_produto(produtos: list):

    if not produtos:
        print("A lista de produtos está vazia.")
        return None

    print("Selecione um produto:")
    for indice, produto in enumerate(produtos):
        print(f"{indice + 1}. {produto}")

    while True:
        try:
            print("--------------------------------------------------------------")
            escolha = int(input("Digite o número do produto desejado: "))
            if 1 <= escolha <= len(produtos):
                return produtos[escolha - 1]
            else:
                print("Seleção inválida. Por favor, digite um número da lista.")
        except ValueError:
            print("Entrada inválida. Por favor, digite um número.")

## Agente 2 - Como descartar

In [ ]:
def como_descartar(produto_selecionado: str):

  buscar = Agent(
      name = "como_descartar",
      model = modelo,
      instruction="""
      Você é um assistente de pesquisa especializado em sustentabilidade.
      Sua tarefa é usar a ferramenta de busca do Google (google_search) para encontrar e listar
      a forma correta de descarte para o produto fornecido.
      Apresente somente as instruções de descarte do produto selecionado em bullet points claros e concisos.
      Priorize informações de fontes confiáveis e especialistas da área.
      Se houver múltiplas formas de descarte, mencione as mais recomendadas.
      """,
      description="Agente que busca informações de descarte no Google",
      tools=[google_search]
  )

  forma_descarte = f"Como descartar corretamente o seguinte item: {produto_selecionado}?"

  descarte = call_agent(buscar, forma_descarte)
  return descarte

## Agente 3 - Locais de Descarte e Contatos


In [ ]:
def contato_local(produto_selecionado: str, minha_localizacao: str):

  local = Agent(
      name = "contato_local",
      model = modelo,
      instruction=f"""
      Você é um assistente pessoal. Sua tarefa é, usando a ferramenta de pesquisa do Google (google_search),
      encontrar local de coleta ou descarte para o '{produto_selecionado}', considerando a seguinte localização: '{minha_localizacao}'.
      Liste informações de contato desses locais.
      Para cada local encontrado, forneça de forma sucinta (se disponível na pesquisa):
      - Nome do Local:
      - Endereço:
      - Telefone:
      - Horário de funcionamento:
      Se informações detalhadas não estiverem disponíveis para todos os campos, forneça o que encontrar.
      Priorize informações de contato direto.
      """,
      description="Agente que busca o contato dos locais de coleta no Google",
      tools=[google_search]
  )

  contato_coleta = (
       f"Liste informações de contato (nome, endereço, telefone, horário) de locais "
       f"que coletam '{produto_selecionado}' em '{minha_localizacao}'."
  )

  contato = call_agent(local, contato_coleta)
  return contato

In [ ]:
print("--------------------------------------------------------------")
print("\n♻️ Bem-vindo ao Sistema de Descarte Consciente ♻️\n")
print("--------------------------------------------------------------")

# Inserir lógica do sistema de agentes ################################################
produto_selecionado = selecionar_produto(produtos)
print("--------------------------------------------------------------")

if produto_selecionado:
    print(f"\nVocê quer descartar: {produto_selecionado}\n")
    print("--------------------------------------------------------------")

    instrucao = como_descartar(produto_selecionado)
    print("\n--- 🗑️ Instruções para descarte correto ---\n")
    display(to_markdown(instrucao))
    print("--------------------------------------------------------------")

    ligue = contato_local(produto_selecionado, minha_localizacao)
    print("\n--- 📍  Na sua cidade temos esses locais de coleta  ---\n")
    display(to_markdown(ligue))
    print("--------------------------------------------------------------")

else:
    print("Você não escolheu um produto válido.")
    print("--------------------------------------------------------------")